In [1]:
!uv sync
%load_ext autoreload
%autoreload 2

Resolved 153 packages in 6ms
Audited 150 packages in 0.50ms


In [2]:
import pathlib
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import data_exploration.weather_stations as ws
import data_exploration.viz as viz

In [3]:
folder = pathlib.Path('../data/station_data_parquet/')
flist = list(folder.glob('*.parquet'))
df = pd.concat([ws.read_parquet(f) for f in flist[10:50]], axis=0)

2025-03-12 13:05:42.062 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU3786-1-1.parquet
2025-03-12 13:05:42.127 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU1523-4-1.parquet
2025-03-12 13:05:42.141 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU371-A3-1.parquet
2025-03-12 13:05:42.155 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFM00068490.parquet
2025-03-12 13:05:42.166 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU184-A3-1.parquet
2025-03-12 13:05:42.180 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFM00068848.parque

In [4]:
def get_unique_latlon(df):
    
    lat = df['Latitude'].mean()
    lon = df['Longitude'].mean()
    df = df.assign(lat_unique=lat, lon_unique=lon)

    return df

df = df.groupby('Station_name', as_index=False).apply(get_unique_latlon)

In [5]:
df['time'] = pd.to_datetime(df.index.to_frame().datetime)
year = df.time.dt.year.values

df['x'] = df['lon_unique'].round(2)
df['y'] = df['lat_unique'].round(2)

df = df[year >= 2015]

/Users/luke/Work/Students/benedict_ejelonu/data-exploration/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/luke/Work/Students/benedict_ejelonu/data-exploration/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/luke/Work/Students/benedict_ejelonu/data-exploration/.venv/lib/python3.11/site-packages/geopandas/geodataframe.py:181

In [6]:
ds_stations_gridded = df.reset_index().set_index(['y', 'x', 'time']).to_xarray()

In [7]:
# modis data saved as a template for coordinates 
fname_modis = '../data/tmp.nc'
da_modis = xr.open_dataarray(fname_modis).isel(band=0, time=0, drop=True)

In [8]:
ds_stations_gridded.isel(time=slice(0, 100)).reindex_like(da_modis)

<xarray.Dataset> Size: 4GB
Dimensions:                                   (y: 134, x: 171, time: 100)
Coordinates:
  * y                                         (y) float64 1kB -33.15 ... -34.48
  * x                                         (x) float64 1kB 17.89 ... 19.59
  * time                                      (time) datetime64[ns] 800B 2015...
Data variables: (12/243)
    level_0                                   (y, x, time) float64 18MB nan ....
    Station_name                              (y, x, time) object 18MB nan .....
    datetime                                  (y, x, time) datetime64[ns] 18MB ...
    Station_ID                                (y, x, time) object 18MB nan .....
    Year                                      (y, x, time) float64 18MB nan ....
    Month                                     (y, x, time) float64 18MB nan ....
    ...                                        ...
    remarks_Report_Type                       (y, x, time) float64 18MB nan ....
    remarks_Source_Code                       (y, x, time) float64 18MB nan ....
    remarks_Source_Station_ID                 (y, x, time) object 18MB nan .....
    geometry                                  (y, x, time) object 18MB nan .....
    lat_unique                                (y, x, time) float64 18MB nan ....
    lon_unique                                (y, x, time) float64 18MB nan ....